In [8]:
###Import required packages
from models import loss_biological, toy_model
import numpy as np
from hyperopt import hp, fmin, tpe
from scikits.odes.ode import ode
import pandas as pd
import time

In [9]:
###Specify if data is to be saved to CSV
save_data = False

In [10]:
###Helper function for visualization
def name_converter(A):
    if A == ((0, 1, 0), (1, 0, 0)):
        return 'Dual Control'

    elif A == ((0, 0, 1), (0, 0, 1)):
        return 'No Control'

    elif A == ((0, 0, 1), (1, 0, 0)):
        return 'Downstream Activation'

    elif A == ((0, 1, 0), (0, 0, 1)):
        return 'Upstream Repression'
    else: return 'Invalid Circuit'
    
###Search space definition
space = hp.choice('architecture', 
    [([[0, 0, 1], [1, 0, 0]], [[2., hp.uniform('theta1_da', 0.001, 10), hp.uniform('k1_da', 1E-7, 1E-3)], [2., hp.uniform('theta2_da', 0.001, 10), hp.uniform('k2_da', 1E-7, 1E-3)]]),
     ([[0, 1, 0], [0, 0, 1]], [[2., hp.uniform('theta1_ur', 0.001, 10), hp.uniform('k1_ur', 1E-7, 1E-3)], [2., hp.uniform('theta2_ur', 0.001, 10), hp.uniform('k2_ur', 1E-7, 1E-3)]]),
     ([[0, 0, 1], [0, 0, 1]], [[2., hp.uniform('theta1_nc', 0.001, 10), hp.uniform('k1_nc', 1E-7, 1E-3)], [2., hp.uniform('theta2_nc', 0.001, 10), hp.uniform('k2_nc', 1E-7, 1E-3)]]),
     ([[0, 1, 0], [1, 0, 0]], [[2., hp.uniform('theta1_dc', 0.001, 10), hp.uniform('k1_dc', 1E-7, 1E-3)], [2., hp.uniform('theta2_dc', 0.001, 10), hp.uniform('k2_dc', 1E-7, 1E-3)]])])
    

###Objective function
def run_hyperopt(max_iters):
    losses = []
    params = []
    circuits = []

    #Define objective function
    def objective(args):
        architecture, param_values = args
        #Integration conditions
        t = np.linspace(0, 5E4, 100) 
        y0 = np.array([2290., 0., 0., 0., 0., 0.])

        extra_options = {'old_api': False, 'user_data': [architecture, param_values]}
        ode_solver = ode('cvode', toy_model, **extra_options)
        solution = ode_solver.solve(t, y0)
        j1, j2 = solution.values.y[-1, -2:]
        j1, j2, loss = loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2)

        losses.append(loss)
        params.append(param_values)
        circuits.append(architecture)
        return loss

    #Run hyperopt call
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_iters)
    #Create trajectory data frame
    landscape = pd.DataFrame({'Circuit':circuits, 'Loss': losses, 'k1': [params[i][0][2] for i in range(len(params))], 'k2': [params[i][1][2] for i in range(len(params))], 'theta1': [params[i][0][1] for i in range(len(params))], 'theta2': [params[i][1][1] for i in range(len(params))]})
    landscape['Circuit'] = [name_converter(c) for c in landscape.Circuit]

    landscape = landscape.reset_index()

    best_loss = 1E5
    best_circuit = 'Initial'
    best_losses = []
    best_losses_circuits = []
    for i in range(len(landscape)):
        if landscape.Loss[i] < best_loss:
            best_loss = landscape.Loss[i]
            best_circuit = landscape.Circuit[i]
        best_losses.append(best_loss)
        best_losses_circuits.append(best_circuit)
    landscape['best_losses'] = best_losses
    landscape['best_loss_circuit'] = best_losses_circuits
    return landscape, best

In [ ]:
###Run single architecture search to get sample run
max_iters = 500
landscape, best = run_hyperopt(max_iters)
if save_data: landscape.to_csv('../data/toy_model_sample_run.csv')

In [ ]:
###Run hyperopt 100 times to solve for background
max_iters = 500
total_background =  pd.DataFrame()
for i in range(100):
    landscape, best = run_hyperopt(max_iters)
    total_background = pd.concat([landscape, total_background])
    #background.to_csv('background.csv', mode='a')
    
if save_data: total_background.to_csv('../data/toy_model_background.csv')

In [ ]:
# #Run single forward optimization grid search to create landscapes for figure
# architectures = [((0, 0, 1), (1, 0, 0)), ((0, 1, 0), (0, 0, 1)), ((0, 0, 1), (0, 0, 1)), ((0, 1, 0), (1, 0, 0))]
# k1s = np.linspace(1E-7, 1E-3, 10)
# k2s = np.linspace(1E-7, 1E-3, 10)
# theta1s = np.linspace(0.001, 10, 10)
# theta2s = np.linspace(0.001, 10, 10)
# t = np.linspace(0, 5E4, 100) 
# y0 = np.array([2290., 0., 0., 0., 0., 0.]) 

# landscape_grid = pd.DataFrame()
# for architecture in architectures:
#         start_time = time.time()
#         for k1 in k1s:
#                 # for k2 in k2s:
#                         for theta1 in theta1s:
#                                 for theta2 in theta2s:
#                                         param_values = [[2., theta1, k1], [2., theta2, k2]]
#                                         extra_options = {'old_api': False, 'user_data': [architecture, param_values]}
#                                         ode_solver = ode('cvode', toy_model, **extra_options)
#                                         solution = ode_solver.solve(t, y0)
#                                         j1, j2 = solution.values.y[-1, -2:]
#                                         j1, j2, loss = loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2)
#                                         test = pd.DataFrame({'index':[0], 'architecture':name_converter(architecture), 'theta1':theta1, 'theta2':theta2, 'k1':k1, 'k2':k2, 'loss':loss})
#                                         landscape_grid = pd.concat([landscape_grid, test])
#         elapsed_time = time.time() - start_time
#         landscape_grid['time'] = elapsed_time
#         print(architecture, 'architecture complete')
# if save_data: landscape_grid.to_csv('../data/toy_model_landscapes_grid_search.csv')